In [2]:
import duckdb
con = duckdb.connect("c:/Code/Local Code/deadlock_match_prediction/data/deadlock.db")



In [3]:
df = con.execute("SELECT * FROM player_matches LIMIT 10").fetchdf()
df

,account_id,match_id,hero_id,team,kills,deaths,assists,denies,net_worth
0,184693821,35508341,60,Team0,6,14,20,3,34242
1,341420696,35508341,31,Team0,10,7,16,4,38354
2,361839040,35508341,50,Team0,14,9,16,4,41878
3,69587081,35508341,15,Team1,7,14,12,4,38376
4,1845214321,35508341,58,Team1,13,9,4,10,46643
5,247872508,35508341,11,Team1,4,12,11,14,39900
6,1731846884,35508341,20,Team0,7,7,28,5,45976
7,174042857,35508341,19,Team1,14,8,13,5,48611
8,386039958,35508341,25,Team1,5,10,18,3,43303
9,202107919,35508341,12,Team0,3,10,25,5,34765


In [ ]:
response = con.execute("""SELECT 
    account_id,
    COUNT(DISTINCT match_id) AS match_count
FROM 
    player_matches
GROUP BY 
    account_id
ORDER BY 
    match_count DESC
LIMIT 25;""").fetchdf()
print(response)

    account_id  match_count
0    483770846           68
1    291902996           53
2   1425284998           49
3    130174860           49
4   1026126614           41
5    408877348           39
6    386039958           38
7     34928102           38
8   1062898121           37
9    409056238           36
10   140062687           35
11    97506421           34
12   137322787           34
13   951692856           34
14   204288555           33
15    87241154           33
16   356216207           33
17  1192612644           32
18   384529372           32
19  1754045322           32
20   133308331           31
21    97652111           31
22   165560339           31
23  1868839452           31
24   223305242           31
25   120623629           31
26   349804083           30
27  1659551328           30
28   385814004           29
29   136898005           29
30  1186801441           29
31    84379844           29
32  1165470690           28
33  1475821079           28
34   122953761      

In [14]:
response = con.execute("""SELECT 
    match_count,
    COUNT(*) AS player_count
FROM (
    SELECT 
        account_id,
        COUNT(DISTINCT match_id) AS match_count
    FROM 
        player_matches
    GROUP BY 
        account_id
) t
GROUP BY 
    match_count
ORDER BY 
    match_count;""").fetchdf()
print(response)

    match_count  player_count
0             1           880
1             2           311
2             3           129
3             4            95
4             5            60
5             6            41
6             7            31
7             8            25
8             9            16
9            10            12
10           11             9
11           12            11
12           13             5
13           14             6
14           15             4
15           16             3
16           17             3
17           18             2
18           19             1
19           20             2
20           21             1
21           22             2
22           31             1
23           34             1


In [19]:
response = con.execute("""
        SELECT 
            pm.account_id,
            COUNT(DISTINCT pm.match_id) AS match_count,
            MAX(m.start_time) AS match_time
        FROM 
            player_matches pm
        JOIN
            matches m ON pm.match_id = m.match_id
        GROUP BY 
            pm.account_id
        HAVING 
            match_count > 10
        ORDER BY
            match_time DESC
        LIMIT 5
        """).fetchdf()
print(response)

   account_id  match_count          match_time
0   165560339           14 2025-05-05 23:55:31
1    97506421           19 2025-05-05 23:55:31
2   472792201           11 2025-05-05 23:55:31
3   924812291           15 2025-05-05 23:52:42
4   291902996           20 2025-05-05 23:50:05
